# Find best hyperparameters

In [1]:
def evaluation_function(parameters, datasets, n_samples=5):
    """
    Test the ESN with specific parameters on NARMA-10
    :param parameters: Dictionary with parameters values
    :param datasets: The dataset for the evaluation
    :param n_samples: How many samples to test the model ?
    :return: A tuple (model, fitness value)
    """
    # Batch size (how many sample processed at the same time?)
    batch_size = 1

    # Predicted/target plot length
    plot_length = 200

    # Use CUDA?
    use_cuda = False
    use_cuda = torch.cuda.is_available() if use_cuda else False

    # Reservoir hyper-parameters
    spectral_radius = parameters['spectral_radius']
#     leaky_rate = parameters['leaky_rate']
    input_dim = 1
    reservoir_size = parameters['reservoir_size']
    connectivity = parameters['connectivity']
    ridge_param = parameters['ridge_param']
    input_scaling = parameters['input_scaling']
    bias_scaling = parameters['bias_scaling']

    # Data loader
    trainloader = DataLoader(datasets[0], batch_size=batch_size, shuffle=False)
    testloader = DataLoader(datasets[1], batch_size=batch_size, shuffle=False)

    # Average NRMSE
    NRMSE_average = 0.0

    # For each samples
    for n in range(n_samples):
        # Internal matrix
        w_generator = echotorch.utils.matrix_generation.NormalMatrixGenerator(
            connectivity=connectivity,
            spetral_radius=spectral_radius
        )

        # Input weights
        win_generator = echotorch.utils.matrix_generation.NormalMatrixGenerator(
            connectivity=connectivity,
            scale=input_scaling,
            apply_spectral_radius=False
        )

        # Bias vector
        wbias_generator = echotorch.utils.matrix_generation.NormalMatrixGenerator(
            connectivity=connectivity,
            scale=bias_scaling,
            apply_spectral_radius=False
        )

        # Create a Leaky-integrated ESN,
        # with least-square training algo.
        # esn = etrs.ESN(
        esn = etrs.ESN(
            input_dim=input_dim,
            hidden_dim=reservoir_size,
            output_dim=1,
            learning_algo='inv',
            w_generator=w_generator,
            win_generator=win_generator,
            wbias_generator=wbias_generator,
            ridge_param=ridge_param
        )

        # Transfer in the GPU if possible
        if use_cuda:
            esn.cuda()
        # end if

        # For each batch
        for data in trainloader:
            # Inputs and outputs
            inputs, targets = data

            # Transform data to Variables
            inputs, targets = Variable(inputs), Variable(targets)
            if use_cuda: inputs, targets = inputs.cuda(), targets.cuda()

            # ESN need inputs and targets
            esn(inputs, targets)
        # end for

        # Now we finalize the training by
        # computing the output matrix Wout.
        esn.finalize()

        # Get the first sample in test set,
        # and transform it to Variable.
        dataiter = iter(testloader)
        test_u, test_y = dataiter.next()
        test_u, test_y = Variable(test_u), Variable(test_y)
        if use_cuda: test_u, test_y = test_u.cuda(), test_y.cuda()

        # Make a prediction with our trained ESN
        y_predicted = esn(test_u)

        # Add to sum of NRMSE
        NRMSE_average += echotorch.utils.nrmse(y_predicted.data, test_y.data)
    # end for

    # Print test MSE and NRMSE
    return esn, NRMSE_average / n_samples
# end evaluation_function


In [2]:
import echotorch.utils.optimization as optim
import numpy as np
import json
from narma_evaluation import evaluation_function
from utils import *

def find_hyperparams(prediction_horizon, interval):
    # Manual seed initialisation
    np.random.seed(1)
    torch.manual_seed(1)

    # Get a random optimizer
    random_optimizer = optim.optimizer_factory.get_optimizer('random', R=50)

    # Parameters ranges
    param_ranges = dict()
    param_ranges['spectral_radius'] = np.arange(0.1, 1.1, 0.1)
    # param_ranges['leaky_rate'] = np.arange(0.1, 1.1, 0.1)
    param_ranges['reservoir_size'] = np.arange(50, 500, 50)
    param_ranges['connectivity'] = np.arange(0.1, 1.0, 0.1)
    param_ranges['ridge_param'] = np.logspace(-10, 2, base=10, num=10)
    param_ranges['input_scaling'] = np.arange(0.1, 1.1, 0.1)
    param_ranges['bias_scaling'] = np.arange(0.0, 1.1, 0.1)


    # data parameters
    window_size = 1
#     prediction_horizon = 5

    ds_builder = DatasetBuilder()
    X_train, y_train, X_test, y_test = ds_builder.build_dataset_simple(prediction_horizon, interval, data_path='../data/input_data.csv', train_pct=0.8)

    dataset_train = MyDataset(X_train, y_train)
    dataset_test = MyDataset(X_test, y_test)

    # Launch the optimization of hyper-parameters
    _, best_param, best_NRMSE = random_optimizer.optimize(
        evaluation_function,
        param_ranges,
        (dataset_train, dataset_test),
        n_samples=5
    )

    # Show the result
    print(f"Best hyper-parameters found for interval {interval}: {best_param}")
    print(f"Best NRMSE : {best_NRMSE}")

    for k, v in best_param.items():
        best_param[k] = float(v)
        
    with open(f'hyperparams/hyperparams_esn_{interval}.json', 'w') as fp:
        json.dump(best_param, fp)

In [3]:
prediction_horizons = [5, 8, 7]
interval = ['1min', '60min', 'daily']

for prediction_horizon, interval in zip(prediction_horizons, interval):
    find_hyperparams(prediction_horizon, interval)
    

/Users/amalota/Documents/magisterka/ml/project/time-series-prediction-using-reservoir-computing/models/utils.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X, y = torch.tensor(X), torch.tensor(y)


Best hyper-parameters found for interval 1min: {'spectral_radius': 0.30000000000000004, 'reservoir_size': 300, 'connectivity': 0.30000000000000004, 'ridge_param': 2.1544346900318823e-05, 'input_scaling': 0.2, 'bias_scaling': 1.0}
Best NRMSE : 0.2634685919495313
Best hyper-parameters found for interval 60min: {'spectral_radius': 0.5, 'reservoir_size': 200, 'connectivity': 0.4, 'ridge_param': 0.00046415888336127724, 'input_scaling': 0.1, 'bias_scaling': 0.5}
Best NRMSE : 0.7020035128000529
Best hyper-parameters found for interval daily: {'spectral_radius': 0.1, 'reservoir_size': 250, 'connectivity': 0.5, 'ridge_param': 2.1544346900318823e-05, 'input_scaling': 0.1, 'bias_scaling': 0.8}
Best NRMSE : 0.6661524030108785
